pip install langchain
pip install nomic embed
ollama pull llama3
ollama list

In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [2]:
local_path = "B:\OneDrive - Amity University\Desktop\Dataset\data\RHP_AWFIS\Awfis Space Solutions Limited - Red Herring Prospectus.pdf"

if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [7]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma


In [5]:
# Spliting and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=75000, chunk_overlap=9000)
chunks = text_splitter.split_documents(data)

# Adding to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 27/27 [16:22<00:00, 36.40s/it]


In [8]:
!pip list

Package                                  Version
---------------------------------------- --------------
absl-py                                  2.1.0
accelerate                               0.30.1
aiofiles                                 23.2.1
aiohttp                                  3.9.5
aiosignal                                1.3.1
alembic                                  1.13.1
altair                                   5.3.0
aniso8601                                9.0.1
annotated-types                          0.6.0
antlr4-python3-runtime                   4.9.3
anyio                                    4.3.0
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
arrow                                    1.3.0
asgiref                                  3.8.1
asttokens                                2.4.1
astunparse                               1.6.3
async-lru                                2.0.4
attrs                                    23.

In [9]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough



In [12]:
from langchain.retrievers.multi_query import (MultiQueryRetriever)

In [13]:
local_model = "llama3"
llm = ChatOllama(model=local_model)


QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to provide one version
    of the given user's question to get relevant documents from
    a vector database. By giving simliar perspective on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search.Provide these alternative questions separated by newlines.
    Original question: {question}""",
)


RAG combining WITH LLAMA3

In [14]:
retriever= MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT,
    
)


template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt1 = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt1
    | llm
    | StrOutputParser()
)






GIVING QUESTION TO LLM-RAG MODEL


In [15]:
try:
    output =chain.invoke("which company is providing this RED HERRING PROSPECTUS and why?")
    print("Response:",output)

except Exception as e:
    print("An error occurred:", e)


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.16s/it]


Response: Based on the provided context, it appears that Awfis Space Solutions Limited is the company providing this Red Herring Prospectus (RHP). A Red Herring Prospectus is a preliminary document filed with regulatory bodies, such as the Securities and Exchange Board of India (SEBI), by companies planning to make an initial public offering (IPO) or offer shares for sale. The RHP provides detailed information about the company's financials, management, business model, and other relevant details to prospective investors.

In this case, Awfis Space Solutions Limited is preparing to make an IPO and has filed this Red Herring Prospectus with SEBI as part of the regulatory process.


In [17]:
try:
    user_question=input("Enter your Question please:")
    output =chain.invoke(user_question)
    print("You have the asked the Question: ",user_question)
    print("Response:",output)

except Exception as e:
    print("An error occurred:", e)


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


You have the asked the Question:  the awfis company making profit or loss? and also what about the assets and liabilities of the company?
Response: Based on the provided context, here are some key points:

**Profit/Loss:** The company has delivered returns to the space owner of ₹62.71 per sq. ft. on chargeable area during Fiscal 2023, which is 14.02% higher than the monthly office rentals for the same micro-market. This suggests that the company is making a profit.

**Assets:** There is no direct information provided about the company's assets in this context. However, we can infer some information:

* The company has a large and diversified vendor base with over 350 vendors across India.
* It has a skilled team of 63 designers and project managers as on December 31, 2023.

**Liabilities:** Again, there is no direct information provided about the company's liabilities. However, we can make some inferences:

* The company has rate contracts with vendors to provide various services, whic